In [3]:
import torch
print(torch.__version__)

1.7.1


In [4]:
from __future__ import print_function

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision import models
import numpy
import dataset
import random
from utils import *
from VGG_gru import FERANet
# from tensorboardX import SummaryWriter
import argparse
import subprocess
import math

In [5]:
# os.environ['CUDA_VISIBLE_DEVICES'] = "1"

description='PyTorch Facial Expression'
batch_size = 1
epochs = 20 #120
length = 32
preInitial = True
# dataset = 'Emotiw'

In [6]:
trn= "BDItrain1.txt"
vl= "BDIval1.txt"
tst= "BDItest1.txt"

In [7]:
trainlist = "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+trn
vallist =  "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+vl
testlist =  "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+tst

In [8]:
print(trainlist)

/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/BDItrain1.txt


In [9]:
path_d= "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/dataset.py"

In [10]:
if os.path.isfile != True:
	subprocess.call(["python", path_d])

In [11]:
# if os.path.isfile != True:
# 	subprocess.call(["python", "./TrainTestlist/"+dataset+"/getTraintest_"+dataset+".py"])

In [12]:
backupdir     = "/tmp/duaa/GRU"
batch_size    = 1
# learning_rate = 0.00001
learning_rate = 0.0001

# accuracy = 0.
# best_accuracy = 0.
# metric = AccumulatedAccuracyMetric()
metric = RegressionMetric()

####here for same result#####
num_workers   = 0
# torch.backends.cudnn.enabled = False
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.deterministic = True

In [13]:
if preInitial == True:
	model = FERANet()
	model = Initial(model)
else:
	model = FERANet()

start loading vgg16 pre-fera2013 model...
finish loading vgg16 pre-fera2013 model!


In [14]:
processed_batches = 0
kwargs = {'num_workers': num_workers, 'pin_memory': True}
model = model.cuda()


optimizer = optim.SGD(model.parameters(), lr=learning_rate , momentum=0.9, weight_decay= 0.00005)

# loss_function = nn.CrossEntropyLoss()
loss_function = nn.L1Loss()
# loss_function = nn.MSELoss()

use_Tensorboard = False

In [13]:
def train(epoch,optimizer):


	train_loader = torch.utils.data.DataLoader(
		dataset.listDataset(trainlist,length = length,
					   shuffle=True,
					   train=True,
					   dataset = dataset),
		batch_size=batch_size, shuffle=False, **kwargs)


	for param_group in optimizer.param_groups:
		train_learning_rate = float(param_group['lr'])

	logging('epoch %d, processed %d samples, lr %f' % (epoch, epoch * len(train_loader.dataset), train_learning_rate))

	running_loss = 0.0

	model.train()

	for batch_idx, (data, label) in enumerate(train_loader):


		data = data.squeeze(0)
		data = Variable(data).cuda()

# 		label = Variable(label.long()).cuda()
		label = Variable(label.float()).cuda()
# 		label = label.squeeze(1)

		optimizer.zero_grad()

		output = model(data)

		loss = loss_function(output, label)

		running_loss += loss.data

		loss.backward()

		optimizer.step()

	if epoch %1 == 0:
		logging('Train Loss:{:.6f}'.format(running_loss))

In [ ]:
# 	test_loader = torch.utils.data.DataLoader(
# 		dataset.listDataset(vallist,length = length,
# # 		dataset.listDataset(vallist,length = length,        
# 					shuffle=False,
# 					train=False,
# 					dataset = dataset),
# 					batch_size=1, shuffle=False, **kwargs)
    
    

In [14]:
def eval(epoch,metric):

	model.eval()
    
	test_loader = torch.utils.data.DataLoader(
		dataset.listDataset(vallist,length = length,
# 		dataset.listDataset(vallist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)


	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

		label = Variable(label.float(),volatile=True).cuda()
# 		label = Variable(label.long(),volatile=True).cuda()        
# 		label = label.squeeze(1)
# 		print('label')
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])

			output.append(output_feature)

		output = torch.mean(torch.cat(output), 0, keepdim=True)
        
# 		print("Dua's output is: ", output)
		metric(output, label) 
		eval_loss = metric.value()
# # 		print(metric.value())

# 		accuracy, eval_loss = metric.value()
# # 		print("Dua's test is: ",accuracy)

        
# 	if accuracy >= best_accuracy:
# 		best_accuracy = accuracy
# 		print("saving accuracy is: ",accuracy)
# 		torch.save(model.state_dict(),'%s/model_%d.pkl' % (backupdir,epoch))
	if epoch %1 == 0:
		logging("Eval Loss: %f" % (eval_loss))
    
# 	logging("test accuracy: %f" % (accuracy))
# 	logging("best accuracy: %f" % (best_accuracy))

# 	return eval_loss

In [15]:
def test(epoch,metric):

	model.eval()

# 	global best_accuracy
# 	global accuracy

	metric.reset()
    
	accuracy = 0.
	best_accuracy = 0.
    
	test_loader = torch.utils.data.DataLoader(
		dataset.listDataset(testlist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)


	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

# 		label = Variable(label.long(),volatile=True).cuda()
		label = Variable(label.float(),volatile=True).cuda()
		label = label.squeeze(1)
# 		print('label')
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])

			output.append(output_feature)

		output = torch.mean(torch.cat(output), 0, keepdim=True)
        

# 		metric(output, label) 
# # 		print(metric.value())

# 		accuracy, eval_loss = metric.value()
# # 		print(accuracy)

        
# 	if accuracy >= best_accuracy:
# 		best_accuracy = accuracy
# # 		print("saving accuracy is: ",accuracy)
# # 		torch.save(model.state_dict(),'%s/model_%d.pkl' % (backupdir,epoch))

# 	logging("test accuracy: %f" % (accuracy))
# 	logging("best accuracy: %f" % (best_accuracy))

	return accuracy

In [16]:
for epoch in range(1, epochs+1): 

	train(epoch,optimizer)
	with torch.no_grad():
		eval_accuary = eval(epoch,metric)

2021-02-08 02:38:13 epoch 1, processed 36 samples, lr 0.000100
2021-02-08 02:38:26 Train Loss:338.817474


/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


2021-02-08 02:41:18 Eval Loss: 113.439445
2021-02-08 02:41:18 epoch 2, processed 72 samples, lr 0.000100
2021-02-08 02:41:29 Train Loss:335.009094
2021-02-08 02:44:17 Eval Loss: 226.296997
2021-02-08 02:44:17 epoch 3, processed 108 samples, lr 0.000100
2021-02-08 02:44:28 Train Loss:327.431091
2021-02-08 02:47:17 Eval Loss: 338.370636
2021-02-08 02:47:17 epoch 4, processed 144 samples, lr 0.000100
2021-02-08 02:47:27 Train Loss:319.656555
2021-02-08 02:50:16 Eval Loss: 449.360596
2021-02-08 02:50:16 epoch 5, processed 180 samples, lr 0.000100
2021-02-08 02:50:26 Train Loss:317.020386
2021-02-08 02:53:13 Eval Loss: 559.303162
2021-02-08 02:53:13 epoch 6, processed 216 samples, lr 0.000100
2021-02-08 02:53:22 Train Loss:310.052246
2021-02-08 02:56:09 Eval Loss: 668.582947
2021-02-08 02:56:09 epoch 7, processed 252 samples, lr 0.000100
2021-02-08 02:56:19 Train Loss:301.695099
2021-02-08 02:59:07 Eval Loss: 777.199158
2021-02-08 02:59:07 epoch 8, processed 288 samples, lr 0.000100
2021-02

In [ ]:
# torch.save(model.state_dict(), PATH)

In [ ]:
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()

In [17]:
# for epoch in range(1, epochs+1): 
# 	with torch.no_grad():
# 		eval_accuary = test(epoch,metric)

In [ ]:
# def image_loader(loader, image_name):
#     image = Image.open(image_name)
#     image = loader(image).float()
#     image = torch.tensor(image, requires_grad=True)
#     image = image.unsqueeze(0)
#     return image

# data_transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor()
# ])


# model_ft = models.resnet152(pretrained=True)
# model_ft.eval()

# print( np.argmax(model_ft(image_loader(data_transforms, $FILENAME)).detach().numpy()))


In [ ]:
# def unique_file(basename, ext):
#     actualname = "%s.%s" % (basename, ext)
#     c = itertools.count()
#     while os.path.exists(actualname):
#         actualname = "%s (%d).%s" % (basename, next(c), ext)
#     return actualname
# #usage:
# for i in range(5):
#     with open(unique_file("foo", "txt"), "w") as f:
#         f.write(str(i))